
scheduler index?

ROUTE = /schedule

ROOT = true

METHOD = get


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Schedule a Meeting</title>
  <link href="https://cdn.jsdelivr.net/npm/fullcalendar@3.3.0/dist/fullcalendar.min.css" rel="stylesheet" />
  <link rel="stylesheet" href="/schedule-style.css" />
</head>

<body>

  <div id="calendar"></div>

  <!-- Modal for meeting details -->
  <div id="meetingModal" class="modal">
    <div class="modal-content">
      <h2>Schedule a Meeting</h2>
      <form id="meetingForm">
        <label for="summary">Meeting Summary:</label>
        <input type="text" id="summary" name="summary" required><br>

        <label for="location">Location:</label>
        <input type="text" id="location" name="location" required><br>

        <label for="description">Description:</label>
        <textarea id="description" name="description" required></textarea><br>

        <label for="startDateTime">Start Date and Time:</label>
        <input type="datetime-local" id="startDateTime" name="startDateTime" required><br>

        <label for="endDateTime">End Date and Time:</label>
        <input type="datetime-local" id="endDateTime" name="endDateTime" required><br>

        <label for="email">Your Email:</label>
        <input type="email" id="email" name="email" required><br>

        <button type="submit">Schedule Meeting</button>
      </form>
    </div>
  </div>

  <script src="https://cdnjs.cloudflare.com/ajax/libs/moment.js/2.29.4/moment.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/moment-timezone/0.5.43/moment-timezone-with-data.min.js"></script>
  <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/fullcalendar@3.3.0/dist/fullcalendar.min.js"></script>
  <script src="/schedule-script.js"></script>

</body>

</html>

schedule stylesheet?

ROUTE = /schedule-style.css


In [ ]:
* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

body {
  font-family: 'Roboto', sans-serif;
  background-color: #f1f3f4;
  color: #202124;
  display: flex;
  justify-content: center;
  align-items: flex-start;
  padding-top: 40px;
  min-height: 100vh;
}

/* Calendar Container */
#calendar {
  width: 90%;
  max-width: 1000px;
  margin: 0 auto;
}

/* Modal Overlay */
.modal {
  display: none;
  position: fixed;
  z-index: 999;
  inset: 0;
  background-color: rgba(0, 0, 0, 0.3);
  display: flex;
  justify-content: center;
  align-items: center;
}

/* Modal Content Box */
.modal-content {
  background-color: #fff;
  padding: 30px 25px;
  border-radius: 12px;
  box-shadow: 0 4px 16px rgba(0, 0, 0, 0.2);
  width: 320px;
  max-width: 90%;
  text-align: left;
  animation: fadeIn 0.3s ease-in-out;
}

/* Smooth fade-in */
@keyframes fadeIn {
  from { opacity: 0; transform: scale(0.95); }
  to { opacity: 1; transform: scale(1); }
}

.modal-content h2 {
  font-size: 20px;
  margin-bottom: 20px;
  font-weight: 500;
  color: #202124;
}

/* Labels and Inputs */
label {
  font-size: 14px;
  color: #5f6368;
  margin-top: 15px;
  display: block;
}

input,
textarea {
  width: 100%;
  padding: 10px 12px;
  margin-top: 6px;
  font-size: 14px;
  border: 1px solid #dadce0;
  border-radius: 6px;
  background-color: #fff;
  transition: border-color 0.2s ease-in-out;
}

input:focus,
textarea:focus {
  outline: none;
  border-color: #1a73e8;
  box-shadow: 0 0 0 2px rgba(26, 115, 232, 0.2);
}

/* Submit Button */
button {
  margin-top: 20px;
  padding: 10px 16px;
  background-color: #1a73e8;
  color: white;
  border: none;
  border-radius: 6px;
  font-size: 14px;
  font-weight: 500;
  cursor: pointer;
  width: 100%;
  transition: background-color 0.2s ease-in-out;
}

button:hover {
  background-color: #1669c1;
}

.fc .fc-toolbar>*>* {
  align-items: center;
  display: flex;
}

.fc-toolbar .fc-right {
  display: flex;
}

.fc-toolbar .fc-left {
  flex: 100%;
}

.fc-toolbar.fc-header-toolbar {
  display: flex;
}


schedule client script?

ROUTE = /schedule-script.js


In [ ]:
function toQueryString(obj, prefix = '') {
  const pairs = [];

  for (const key in obj) {
    if (!obj.hasOwnProperty(key)) continue;

    const value = obj[key];
    const paramKey = prefix ? `${prefix}[${encodeURIComponent(key)}]` : encodeURIComponent(key);

    if (value !== null && typeof value === 'object') {
      pairs.push(toQueryString(value, paramKey));
    } else {
      pairs.push(`${paramKey}=${encodeURIComponent(value)}`);
    }
  }

  return pairs.join('&');
}

function convertGoogleEventsToCalendar(events, calendarName) {
  return events.map(e => {
    const eventStart = e.event.start.dateTime || e.event.start.date; // ISO string
    const startDate = new Date(eventStart);

    // Use explicit end if available, else add 1 hour
    const eventEnd = e.event.end?.dateTime || e.event.end?.date;
    const endDate = eventEnd
      ? new Date(eventEnd)
      : new Date(startDate.getTime() + 60 * 60 * 1000);

    return {
      title: e.event.summary || 'No Title',
      start: startDate,
      end: endDate,
      description: e.event.description || '',
      location: e.event.location || '',
      id: e.event.id,
      calendarName: calendarName
    };
  });
}

const PUBLIC_CALENDARS = [
  'Diet', 'General', 'Revelation', 'Robot do', 'Predictions'
]

$(document).ready(function () {
  // Initialize FullCalendar
  $('#calendar').fullCalendar({
    events: async function loadEvents(timeMin, timeMax, timezone, callback) {
      try {
        const query = toQueryString({
          options: {
            timeMin: timeMin.toISOString(),
            timeMax: timeMax.toISOString(),
            timezone
          }
        });

        const calendarResponse = await fetch('/rpc/list%20google%20calendars');
        const calendars = await calendarResponse.json();
        let formattedEvents = []
        for (let i = 0; i < calendars.length; i++) {
          if(!PUBLIC_CALENDARS.includes(calendars[i].summary)) {
            continue
          }
          const response = await fetch('/rpc/list%20events?t=' + Date.now() +'&options[calendarId]=' + calendars[i].id + '&' + query.toString());
          const data = await response.json();
          formattedEvents = formattedEvents.concat(convertGoogleEventsToCalendar(data, calendars[i].summary))
        }
        callback(formattedEvents);
      } catch (err) {
        console.error('Error loading events:', err);
      }
    }, // Add your events here if needed
    dayClick: function (date, jsEvent, view) {

      // Open the modal
      $('#meetingModal').show();

      const start = new moment(date.toISOString()).startOf('day').format('YYYY-MM-DDTHH:mm')
      const end = new moment(date.toISOString()).endOf('day').format('YYYY-MM-DDTHH:mm')

      // Set start and end datetime fields correctly without mutating the original
      $('#startDateTime').val(start);
      $('#endDateTime').val(end);

      //$('#meetingForm').data('start', start);
      //$('#meetingForm').data('end', end);

    },
    eventRender: function (event, element) {
      switch (event.calendarName) {
        case 'Brian Cullinan':
          element.css('background-color', '#1b73e8');
          break;
        case 'B&B':
          element.css('background-color', '#b1365f');
          break;
        case 'Birthdays':
          element.css('background-color', '#33b679');
          break;
        case 'Bookmarks':
          element.css('background-color', '#f6c026');
          break;
        case 'Calls':
          element.css('background-color', '#f4511e');
          break;
        case 'Diet':
          element.css('background-color', '#d50000');
          break;
        case 'Emotions':
          element.css('background-color', '#616161');
          break;
        case 'Family':
          element.css('background-color', '#f6bf26');
          break;
        case 'General':
          element.css('background-color', '#0b8043');
          break;
        case 'Iga':
          element.css('background-color', '#8e24aa');
          break;
        case 'Predictions':
          element.css('background-color', '#5f6368');
          break;
        case 'Revelation':
          element.css('background-color', '#f4511e');
          break;
        case 'Robot do':
          element.css('background-color', '#3f51b5');
          break;
        case 'Tasks':
          element.css('background-color', '#7986cb');
          break;
        case 'Therapy':
          element.css('background-color', '#039be5');
          break;
        default:
          element.css('background-color', '#9e9e9e'); // fallback/default color
          break;
      }
    },
  });

  $(document).on('click', '#meetingModal', function (e) {
    if (!$(e.target).closest('.modal-content').length) {
      $('#meetingModal').hide(); // or use fadeOut()
    }
  });

  // Handle form submission
  $('#meetingForm').submit(function (event) {
    event.preventDefault(); // Prevent the default form submission

    // Gather form data
    var formData = {
      email: $('#email').val(),
      summary: $('#summary').val(),
      location: $('#location').val(),
      description: $('#description').val(),
      startDateTime: $('#startDateTime').val(),
      clientTimezone: moment.tz.guess(),
      endDateTime: $('#endDateTime').val(),
    };

    // Send the data via POST request
    $.ajax({
      url: '/schedule', // Your backend endpoint
      method: 'POST',
      contentType: 'application/json',
      data: JSON.stringify(formData),
      success: function (response) {
        // Handle successful response
        alert('Meeting scheduled successfully!');
        $('#meetingModal').hide(); // Close the modal
        $('#meetingForm')[0].reset(); // Reset the form
      },
      error: function (error) {
        // Handle error
        alert('Failed to schedule the meeting. Please try again.');
        console.error('Error:', error);
      }
    });
  });
});



schedule service?

ROUTE = /schedule

METHOD = post


In [ ]:
const moment = require('moment')
const authorizeCalendar = importer.import('authorize google calendar'); // Authorization function

async function createEvent(email, summary, location, description, startDateTime, endDateTime, clientTimezone) {
  let calendar;
  try {
    // Authorize and get the calendar service
    calendar = await authorizeCalendar();
    
    // Prepare event object
    const event = {
      summary: summary,
      location: location,
      description: description,
      start: {
        dateTime: moment(startDateTime).format('YYYY-MM-DDTHH:mm:ss'),
        timeZone: clientTimezone
      },
      end: {
        dateTime: moment(endDateTime).format('YYYY-MM-DDTHH:mm:ss'),
        timeZone: clientTimezone
      },
      attendees: [
        { email: email }
      ],
      reminders: {
        useDefault: true
      },
    };

    // Insert event into the calendar
    const response = await calendar.events.insert({
      calendarId: 'primary', // Can be customized to other calendars
      resource: event,
    });

    console.log('Event created: ', response.data);
    return response.data;
  } catch (error) {
    console.error('Error creating event: ', error);
    throw error;
  }
}

module.exports = createEvent



list google calendars?


In [ ]:
const util = require('util')
const authorizeCalendar = importer.import('authorize google calendar'); // Authorization function

async function rpcCalendarList() {
  let calendar = await authorizeCalendar()
  let result = await util.promisify(calendar.calendarList.list.bind(calendar))()
  return (result.data || {}).items || []
}

module.exports = rpcCalendarList
